# Function 6

### First Inspection

The features $\mathbf(x)$ of the black box function $f(\mathbf(x))$ represent six hyperparameters (learning rate, regularisation strength or number of hidden layers, etc.) of an unknown neural network. The goal is to maximimise a perfomance score $y$ yielded by the black-box function $f(\mathbf(x))$. Let us load and inspect the evalutations,

In [1]:
# Depedencies,
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# SKOPT imports,
from skopt import gp_minimize
from skopt.space import Real
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from skopt import Optimizer
from joblib import dump, load

# Loading known evaluations,
X, y = np.load("initial_inputs.npy"), np.load("initial_outputs.npy")

# Inspecting evalutation points,
X

array([[0.27262382, 0.32449536, 0.89710881, 0.83295115, 0.15406269,
        0.79586362],
       [0.54300258, 0.9246939 , 0.34156746, 0.64648585, 0.71844033,
        0.34313266],
       [0.09083225, 0.66152938, 0.06593091, 0.25857701, 0.96345285,
        0.6402654 ],
       [0.11886697, 0.61505494, 0.90581639, 0.8553003 , 0.41363143,
        0.58523563],
       [0.63021764, 0.8380969 , 0.68001305, 0.73189509, 0.52673671,
        0.34842921],
       [0.76491917, 0.25588292, 0.60908422, 0.21807904, 0.32294277,
        0.09579366],
       [0.05789554, 0.49167222, 0.24742222, 0.21811844, 0.42042833,
        0.73096984],
       [0.19525188, 0.07922665, 0.55458046, 0.17056682, 0.01494418,
        0.10703171],
       [0.64230298, 0.83687455, 0.02179269, 0.10148801, 0.68307083,
        0.6924164 ],
       [0.78994255, 0.19554501, 0.57562333, 0.07365919, 0.25904917,
        0.05109986],
       [0.52849733, 0.45742436, 0.36009569, 0.36204551, 0.81689098,
        0.63747637],
       [0.72261522, 0

Despite the sparsity of data points, the domain of the black box function seems to be $[0, 1]^6$.

In [2]:
# Inspecting black-box outputs,
y

array([0.6044327 , 0.56275307, 0.00750324, 0.0614243 , 0.2730468 ,
       0.08374657, 1.3649683 , 0.09264495, 0.0178696 , 0.03356494,
       0.0735163 , 0.2063097 , 0.00882563, 0.26840032, 0.61152553,
       0.01479818, 0.27489251, 0.06676325, 0.04211835, 0.00270147,
       0.01820907, 0.00701603, 0.10050661, 0.47539552, 0.67514163,
       0.51645722, 0.00377748, 0.00313433, 0.02134252, 0.09541116])

The output of the black-box function seem to always be positive and of the similiar magnitude. 

### Optimiser Configuration

The optimisation of ML model, effectively black-box functions, has been studied in literature. A competition by NeurIPS titled _Analysis of the Black-Box Optimization Challenge 2020_ found that BO algorithms which make use of both a surrogate and acquisition function often performed the best. GPR (with RBF and Matern kernels featuring ADA) and RFs were common choices for the creation of the surrogate while LCB/UCB or EI were often paired as the acquistion function. This basic setup was shared by the majority of top BO algorithms. The best performing algoritms employed so called "ensemble methods" where, based on some criteria, the process by which the surrogate model is created was changed and/or the acquistion function. For example...

### Optimiser Initialisation

In [4]:
"""INITALISING THE OPTIMISATION MODEL."""

# Inputting the given evaluations provided by the problem,
X_supplied = X.tolist()
y_supplied = y.tolist()

# Inital kappa value,
initial_kappa = 5

"""OPTIMISER SETTINGS."""

# We define the domain of the black-box function (or the range of the parameter values we want to consider),
space = [Real(0, 1, name="x1"),
         Real(0, 1, name="x2"),
         Real(0, 1, name="x3"),
         Real(0, 1, name="x4"),
         Real(0, 1, name="x5"),
         Real(0, 1, name="x6")
         ]

# Creating the kernel for the GPR,
kernel = ConstantKernel(1.0) * Matern(
    length_scale=(0.1, 0.1, 0.1, 0.1, 0.1, 0.1),
    length_scale_bounds=(1e-2, 1.0),
    nu = 5/2)

# GPR settings,
gpr = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=10
)

# Creating optimisier,
opt = Optimizer(
    dimensions=space,
    base_estimator=gpr,
    acq_func="LCB",
    acq_func_kwargs={"kappa": initial_kappa},
    random_state=0
)

"""CREATING INTIAL OPTIMISER STATE."""

# Supplying given points to optimiser,
opt.tell(X_supplied, (-np.array(y_supplied)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state (zero-th iteration),
dump(opt, "bayes_opt_state_iter0.joblib")

# Printing point query,
print(f"Point Query: {point_query}")

Point Query: [0.0, 0.04010892790215851, 0.6441100449350812, 0.1202679612063733, 0.3558626687628173, 0.8741360875164429]


### Next Query 

In [17]:
def kappa_decay(init_kappa, decay_param, iter_num):
    """Outputs a value of kappa given the current interaction number (current query number)."""
    return init_kappa*np.exp(-(iter_num/decay_param))

In [ ]:
current_query = 1

# Input the new evaluation,
X_new = [[]]
y_new = []

# Loading the previous state of the optimiser,
opt = load(f"bayes_opt_state_iter{current_query - 1}.joblib")

# Updating kappa,
new_kappa = kappa_decay(init_kappa=initial_kappa, decay_param=3.75, iter_num=current_query)
opt.acq_func_kwargs["kappa"] = new_kappa

# Supplying the new query to the optimiser,
opt.tell(X_new, (-np.array(y_new)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state,
dump(opt, f"bayes_opt_state_iter{current_query}.joblib")

# Printing point query,
print(f"Point Query {current_query + 1}: {point_query}")